In [1]:
println("Hello World")

Hello World


In [6]:
i = 1
while i < 1000
     i += 1
    println("iteration", i, "\r")
end

iteration2
iteration3
iteration4
iteration5
iteration6
iteration7
iteration8
iteration9
iteration10
iteration11
iteration12
iteration13
iteration14
iteration15
iteration16
iteration17
iteration18
iteration19
iteration20
iteration21
iteration22
iteration23
iteration24
iteration25
iteration26
iteration27
iteration28
iteration29
iteration30
iteration31
iteration32
iteration33
iteration34
iteration35
iteration36
iteration37
iteration38
iteration39
iteration40
iteration41
iteration42
iteration43
iteration44
iteration45
iteration46
iteration47
iteration48
iteration49
iteration50
iteration51
iteration52
iteration53
iteration54
iteration55
iteration56
iteration57
iteration58
iteration59
iteration60
iteration61
iteration62
iteration63
iteration64
iteration65
iteration66
iteration67
iteration68
iteration69
iteration70
iteration71
iteration72
iteration73
iteration74
iteration75
iteration76
iteration77
iteration78
iteration79
iteration80
iteration81
iteration82
iteration83
iteration84
iteration85


iteration741
iteration742
iteration743
iteration744
iteration745
iteration746
iteration747
iteration748
iteration749
iteration750
iteration751
iteration752
iteration753
iteration754
iteration755
iteration756
iteration757
iteration758
iteration759
iteration760
iteration761
iteration762
iteration763
iteration764
iteration765
iteration766
iteration767
iteration768
iteration769
iteration770
iteration771
iteration772
iteration773
iteration774
iteration775
iteration776
iteration777
iteration778
iteration779
iteration780
iteration781
iteration782
iteration783
iteration784
iteration785
iteration786
iteration787
iteration788
iteration789
iteration790
iteration791
iteration792
iteration793
iteration794
iteration795
iteration796
iteration797
iteration798
iteration799
iteration800
iteration801
iteration802
iteration803
iteration804
iteration805
iteration806
iteration807
iteration808
iteration809
iteration810
iteration811
iteration812
iteration813
iteration814
iteration815
iteration816
iteration817

In [5]:
x = 1
x + 1

2